In [1]:
import wrds 

WRDS Thomson Reuters Mutual Funds: https://wrds-www.wharton.upenn.edu/pages/get-data/thomson-reuters/ 

WRDS CRSP Mutual Funds: https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_q_mutualfunds/ 

Username: lyknguyen 
Password: Khanhly_1609 

In [2]:
conn = wrds.Connection() 

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [ ]:
# conn.list_tables(library='crsp') 
# conn.list_tables(library='tr_mutualfunds') 

In [3]:
# DATA SOURCE 1.1 - CRSP Mutual Funds (fund_summary2)
conn.get_table(library='crsp',table='fund_summary2',obs=5).columns 

Index(['summary_period2', 'crsp_fundno', 'caldt', 'summary_period',
       'nav_latest', 'nav_latest_dt', 'tna_latest', 'tna_latest_dt', 'yield',
       'div_ytd', 'cap_gains_ytd', 'nav_52w_h', 'nav_52w_h_dt', 'nav_52w_l',
       'nav_52w_l_dt', 'unrealized_app_dep', 'unrealized_app_dt', 'asset_dt',
       'per_com', 'per_pref', 'per_conv', 'per_corp', 'per_muni', 'per_govt',
       'per_oth', 'per_cash', 'per_bond', 'per_abs', 'per_mbs', 'per_eq_oth',
       'per_fi_oth', 'maturity', 'maturity_dt', 'cusip8', 'crsp_portno',
       'crsp_cl_grp', 'fund_name', 'ticker', 'ncusip', 'mgmt_name', 'mgmt_cd',
       'mgr_name', 'mgr_dt', 'adv_name', 'open_to_inv', 'retail_fund',
       'inst_fund', 'm_fund', 'index_fund_flag', 'vau_fund', 'et_flag',
       'delist_cd', 'header', 'first_offer_dt', 'end_dt', 'dead_flag',
       'merge_fundno', 'actual_12b1', 'max_12b1', 'mgmt_fee', 'exp_ratio',
       'turn_ratio', 'fiscal_yearend', 'crsp_obj_cd', 'si_obj_cd',
       'accrual_fund', 'sales_restr

In [4]:
# Takes 1 minute to run 
crspfundsum = conn.raw_sql(
    """
    select crsp_fundno,ticker,caldt,crsp_obj_cd,si_obj_cd,mgmt_cd,fund_name,index_fund_flag,retail_fund,inst_fund,lipper_class_name
    from crsp.fund_summary2 
    where caldt <= '2018-12-31'
    order by crsp_fundno, caldt
    """, 
    date_cols=['caldt']
) 

crspfundsum['crsp_fundno'] = crspfundsum['crsp_fundno'].astype(int) # convert to integer 
crspfundsum['year'] = crspfundsum['caldt'].dt.year  # extract the year 

# sort and then duplicates drop fundno and year, keep the last occurrence of each duplicated row 
crspfundsum = crspfundsum.sort_values(by=['crsp_fundno','caldt']).drop_duplicates(subset=['crsp_fundno','year'],keep='last') 